In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('text', usetex=True)
matplotlib.rcParams['text.latex.preamble']=r"\usepackage{amsmath}"


from stuff import *
from utils import *
float_formatter = lambda x: "%.2e" % x
np.set_printoptions(formatter={'float_kind':float_formatter})
plt.rcParams.update({'font.size': 14})
%config InlineBackend.figure_format = 'retina'

from time import time

In [ ]:
iters = 1000
np.random.seed(42)
plt.figure(figsize=(12,5))

model_params = dict(nodes=5, dim=40, B_rank=1)#, graph="erdos-renyi", edge_prob=0.3)
model = Model(**model_params)

mu_x, mu_xy, L_x, L_xy = model.get_mu_L()

t_start = time()
x_res, f_err, cons_err = APDG(iters, model)
print(f"\ntime APDG: {time() - t_start}\n")

plt.subplot(121)
plt.plot(f_err, label='APDG')
plt.title('$|F(\\boldsymbol{x}^k) -F^*|$')
plt.yscale('log')

plt.subplot(122)
plt.plot(cons_err, label='APDG')
plt.title('$||\\boldsymbol{Ax}^k||$')
plt.yscale('log')

t_start = time()
x_res, f_err, cons_err = GDAM(iters, model)
print(f"\ntime GDAM: {time() - t_start}\n")

plt.subplot(121)
plt.plot(f_err, label='Globally Dual')
plt.legend()

plt.subplot(122)
plt.plot(cons_err, label='Globally Dual')
plt.legend()

t_start = time()
x_res, f_err, cons_err = LDAM(iters, model)
print(f"\ntime LDAM: {time() - t_start}\n")

plt.subplot(121)
plt.plot(f_err, label='Locally Dual')
plt.legend()
plt.xlabel('Iteration number')

plt.subplot(122)
plt.plot(cons_err, label='Locally Dual')
plt.legend()
plt.xlabel('Iteration number')

# plt.savefig("n10_d100.eps")

In [ ]:
%%time
iters=4000
repeat = 10 
t_apdm, t_gdam, t_ldam = 0, 0, 0
it_apdm, it_gdam, it_ldam = 0, 0, 0
limit_apdm, limit_gdam, limit_ldam = 0, 0, 0

accuracy = 1e-2

for _ in range(repeat):
    model = Model(**model_params)
    t_start = time()
    x_res, f_err, cons_err = APDG(iters, model, use_crit=True, accuracy=accuracy)
    t_apdm += time() - t_start
    it = (f_err > 0).sum()  # error arrays initialized as np.zeros()
    it_apdm += it
    limit_apdm += (it >= iters)
    print("iters, ", it, "total excessions:", limit_apdm)
    if it >= iters:
        plt.plot(f_err, label="apdm" + str(_))
        plt.yscale("log")
        plt.legend()
        plt.show()
    
    t_start = time()
    x_res, f_err, cons_err = GDAM(iters, model, use_crit=True, accuracy=accuracy)
    t_gdam += time() - t_start
    it = (f_err > 0).sum()  # error arrays initialized as np.zeros()
    it_gdam += it
    limit_gdam += (it >= iters)
    print("iters, ", it, "total excessions:", limit_gdam)
    if it >= iters:
        plt.plot(f_err, label="gdam" + str(_))
        plt.yscale("log")
        plt.legend()
        plt.show()
    
    
    t_start = time()
    x_res, f_err, cons_err = LDAM(iters, model, use_crit=True, accuracy=accuracy)
    t_ldam += time() - t_start
    it = (f_err > 0).sum()  # error arrays initialized as np.zeros()
    it_ldam += it
    limit_ldam += (it >= iters)
    print("iters, ", it, "total excessions:", limit_ldam)
    if it >= iters:
        plt.plot(f_err, label="ldam" + str(_))
        plt.yscale("log")
        plt.legend()
        plt.show()
    
print(f"\ntime APDG (crit): {t_apdm / repeat}, iters: {it_apdm / repeat}, excessions: {limit_apdm}\n")  
print(f"\ntime GDAM (crit): {t_gdam / repeat}, iters: {it_gdam / repeat}, excessions: {limit_gdam}\n")
print(f"\ntime LDAM (crit): {t_ldam / repeat}, iters: {it_ldam / repeat}, excessions: {limit_ldam}\n")
